In [1]:
import numpy as np
import rebound
import matplotlib.pyplot as plt
import matplotlib
import random
import dill
import sys
import pandas as pd
sys.path.append('../generate_training_data')
from training_data_functions import ressummaryfeaturesxgb
%matplotlib inline

folderpath = '/Users/dtamayo/Desktop/MLstability/'
model = 'ressummaryfeaturesxgb_resonantAMD.pkl'

# Load Model

In [2]:
model, features, featurefolder = dill.load(open(folderpath+'/models/'+model, "rb"))

# Make a random simulation as an example

In [3]:
sim = rebound.Simulation()
sim.add(m=1.)
sim.add(m=1.e-5, P=1., e=0.03)
sim.add(m=1.e-5, P=1.2, e=0.03)
sim.add(m=1.e-5, P=1.5, e=0.03)
sim.move_to_com()

# Make the summary features that the model needs to predict. Takes a REBOUND simulation, and a tuple of arguments which shouldn't be changed (These are the values that were used during the training)

In [4]:
%%time
args = (10000, 1000) # (Norbits, Nout) Keep this fixed
summaryfeatures = ressummaryfeaturesxgb(sim, args)

CPU times: user 17.8 s, sys: 22.9 ms, total: 17.8 s
Wall time: 17.4 s


../generate_training_data/training_data_functions.py:408: RuntimeWarning: Degrees of freedom <= 0 for slice
  var = [EM[:j].var() for j in range(len(EM))]
/Users/dtamayo/miniconda3/envs/p3/lib/python3.7/site-packages/numpy/core/_methods.py:110: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/Users/dtamayo/miniconda3/envs/p3/lib/python3.7/site-packages/numpy/core/_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
../generate_training_data/training_data_functions.py:427: RuntimeWarning: Degrees of freedom <= 0 for slice
  var = [Z[:j].var() for j in range(len(Z))]


# We're using the convention that if features = None, then model uses all the features passed back by the function. If features is set, then it will take a subset

In [5]:
if features is not None:
    summaryfeatures = summaryfeatures[features]
summaryfeatures = pd.DataFrame([summaryfeatures]) # convert it to the pandas format model expects

In [11]:
probstability = model.predict_proba(summaryfeatures)[:, 1][0]
print(probstability)

0.0018717946
